In [1]:
import os
import sys
import json
import math
from typing import Dict, Any, List
from dataclasses import dataclass, asdict

import pandas as pd

In [4]:
import os


try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: name 'UserSecretsClient' is not defined


In [10]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


In [11]:
class DataSummary:
    rows: int
    cols: int
    numeric_cols: List[str]
    top_kpis: Dict[str, Any]
    anomalies: List[str]

def summarize_dataframe(df: pd.DataFrame, sample_size:int=5) -> DataSummary:
    rows, cols = df.shape
    numeric_cols = df.select_dtypes("number").columns.tolist()
    top_kpis = {}
    anomalies = []

    # Basic KPIs for numeric columns: mean, median, sum, missing%
    for col in numeric_cols:
        col_ser = df[col].dropna()
        if col_ser.empty:
            continue
        top_kpis[col] = {
            "mean": float(col_ser.mean()),
            "median": float(col_ser.median()),
            "sum": float(col_ser.sum()) if pd.api.types.is_integer_dtype(col_ser) or pd.api.types.is_float_dtype(col_ser) else None,
            "missing_pct": float(df[col].isna().mean() * 100),
            "std": float(col_ser.std()),
        }
        # simple anomaly detection: values > mean + 3*std
        mean, std = top_kpis[col]["mean"], top_kpis[col]["std"]
        if std and col_ser.gt(mean + 3 * std).any():
            anomalies.append(f"High outliers detected in '{col}'")

    return DataSummary(rows=rows, cols=cols, numeric_cols=numeric_cols, top_kpis=top_kpis, anomalies=anomalies)

print("Data Analysis Agent")

Data Analysis Agent


In [12]:
def agent_analyze_data(csv_path: str) -> None:
    df = pd.read_csv(csv_path)
    summary = summarize_dataframe(df)

    # prepare compact context for LLM
    nm = json.dumps(summary.top_kpis, indent=2)[:8000]  # keep it bounded
    system = "You are an enterprise analyst agent. Produce a short list of (1) top 5 insights, (2) 5 practical action items with owners and priority (High/Med/Low), and (3) a 3-line executive summary. Use the provided KPIs and anomalies. Output JSON with keys: executive_summary, insights, action_items."
    user = (
        f"Dataset rows={summary.rows}, cols={summary.cols}, numeric_columns={summary.numeric_cols}.\n"
        f"KPIs (truncated):\n{nm}\n"
        f"Anomalies: {summary.anomalies}\n\n"
        f"Also provide 3 suggested charts to visualize the most important trends (name and short reason)."
    )

    llm_out = call_llm(system, user)
    # print structured result
    try:
        # Some LLM outputs are JSON; attempt to find JSON object inside
        j = json.loads(llm_out)
        print(json.dumps(j, indent=2))
    except Exception:
        # fallback: print as plain text
        print(llm_out)

print("prepare compact context for LLM")

prepare compact context for LLM


In [13]:
class SupportResponse:
    reply: str
    tags: List[str]
    urgency: str
    follow_up: List[str]

def agent_support_reply(ticket_text: str, customer_name: str = None) -> None:
    system = (
        "You are an enterprise customer support agent. Craft a polite, concise reply that (A) empathizes, (B) states what you'll do or need, (C) gives next steps and ETA. "
        "Also return a short list of tags (max 5) and a single urgency: Low / Medium / High. At the end provide 2 follow-up questions to clarify the issue. Output JSON only with keys: reply, tags, urgency, follow_up."
    )
    user = f"Ticket: {ticket_text}\nCustomer name: {customer_name or 'N/A'}\nMake reply maximum 5 sentences. Keep reply friendly and professional."

    llm_out = call_llm(system, user)
    try:
        j = json.loads(llm_out)
        print(json.dumps(j, indent=2))
    except Exception:
        # If LLM didn't output strict JSON, attempt to extract fields heuristically:
        print(llm_out)

print("Support Agent ")

Support Agent 


In [21]:
def usage():
    print(
        "Usage:\n"
        "  python enterprise_agent.py analyze_data <path/to/data.csv>\n"
        "  python enterprise_agent.py support_reply \"<ticket text>\" [customer_name]\n"
    )

def main(argv):
    if len(argv) < 2:
        usage()
        return
    cmd = argv[1]
    if cmd == "analyze_data":
        if len(argv) < 3:
            print("Missing CSV path.")
            return
        agent_analyze_data(argv[2])
    elif cmd == "support_reply":
        if len(argv) < 3:
            print("Missing ticket text.")
            return
        ticket = argv[2]
        customer = argv[3] if len(argv) >= 4 else None
        agent_support_reply(ticket, customer)
    else:
        usage()

if __name__ == "__main__":
    main(sys.argv)

Usage:
  python enterprise_agent.py analyze_data <path/to/data.csv>
  python enterprise_agent.py support_reply "<ticket text>" [customer_name]

